In [ ]:



from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import os
import cv2
import numpy as np
from scipy.ndimage import label
import matplotlib.pyplot as plt
from tqdm import tqdm

# --------------------------
# ABFA Core
# --------------------------

def detect_active_pixels(frame, background_frames, k=3):
    G_mean = np.mean(background_frames, axis=0)
    G_std = np.std(background_frames, axis=0)
    threshold = G_mean + k * G_std
    active = (frame > threshold).astype(np.uint8)
    return active

def connected_components(active_pixels):
    labeled, num_features = label(active_pixels)
    return labeled, num_features

def filter_blobs(labeled_img, sensor_height=2.0):
    min_cells = int(20 / sensor_height)
    max_cells = int(60 / sensor_height)
    valid_count = 0
    for label_id in np.unique(labeled_img):
        if label_id == 0: continue
        count = np.sum(labeled_img == label_id)
        if min_cells <= count <= max_cells:
            valid_count += 1
    return valid_count

def abfa_pipeline(frame, background_frames, sensor_height=2.0):
    active = detect_active_pixels(frame, background_frames)
    labeled, _ = connected_components(active)
    people_count = filter_blobs(labeled, sensor_height)
    return people_count, active, labeled

# --------------------------
 #Dataset Loader
# --------------------------

def load_images_from_folder(folder, max_images=None, resize_shape=(32, 32)):
    images = []
    for filename in sorted(os.listdir(folder)):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
            path = os.path.join(folder, filename)
            img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
            if img is not None:
                img = cv2.resize(img, resize_shape)
                images.append((filename, img))
            if max_images and len(images) >= max_images:
                break
    return images

#  Run on Dataset


dataset_path = "/content/drive/MyDrive/DataCollection"
resize_shape = (32, 32)

# Use some frames from class "1" as background
bg_folder = os.path.join(dataset_path, "1")
background_frames = [img for _, img in load_images_from_folder(bg_folder, max_images=5, resize_shape=resize_shape)]
background_frames = np.array(background_frames)  # shape: (5, 32, 32)

results = []
print("\nRunning ABFA on dataset...\n")

for folder in sorted(os.listdir(dataset_path)):
    folder_path = os.path.join(dataset_path, folder)
    if not os.path.isdir(folder_path): continue
    if not folder.isdigit(): continue

    gt_label = int(folder)
    images = load_images_from_folder(folder_path, resize_shape=resize_shape)

    for filename, frame in tqdm(images, desc=f"Processing class {folder}"):
        pred, _, _ = abfa_pipeline(frame, background_frames)
        results.append((filename, gt_label, pred))

# --------------------------
# 📊 Report Results
# --------------------------

correct = sum(1 for _, gt, pred in results if gt == pred)
total = len(results)

print(f"\n✅ Accuracy: {correct}/{total} = {correct/total:.2%}")

print("\n🔍 Mismatches:")
for filename, gt, pred in results:
    if gt != pred:
        print(f"{filename}: GT={gt}, Pred={pred}")



Running ABFA on dataset...



Processing class 9: 100%|██████████| 304/304 [00:00<00:00, 2215.78it/s]


✅ Accuracy: 559/4052 = 13.80%

🔍 Mismatches:
10.png: GT=1, Pred=0
100.png: GT=1, Pred=0
100_flipped.jpg: GT=1, Pred=0
100_flipped_flipped.jpg: GT=1, Pred=0
100_flipped_flipped_flipped.jpg: GT=1, Pred=0
102.png: GT=1, Pred=0
102_flipped.jpg: GT=1, Pred=0
102_flipped_flipped.jpg: GT=1, Pred=0
102_flipped_flipped_flipped.jpg: GT=1, Pred=0
103.png: GT=1, Pred=2
103_flipped.jpg: GT=1, Pred=2
103_flipped_flipped.jpg: GT=1, Pred=2
103_flipped_flipped_flipped.jpg: GT=1, Pred=2
105.png: GT=1, Pred=5
105_flipped.jpg: GT=1, Pred=2
105_flipped_flipped.jpg: GT=1, Pred=4
105_flipped_flipped_flipped.jpg: GT=1, Pred=2
10_flipped.jpg: GT=1, Pred=2
10_flipped_flipped.jpg: GT=1, Pred=0
10_flipped_flipped_flipped.jpg: GT=1, Pred=2
11.png: GT=1, Pred=0
11_flipped_flipped.jpg: GT=1, Pred=0
12.png: GT=1, Pred=0
12_flipped.jpg: GT=1, Pred=2
12_flipped_flipped.jpg: GT=1, Pred=0
12_flipped_flipped_flipped.jpg: GT=1, Pred=2
13.png: GT=1, Pred=0
13_flipped.jpg: GT=1, Pred=2
13_flipped_flipped.jpg: GT=1, Pred=0
1

In [ ]:
# --------------------------
# 📈 Overall Accuracy
# --------------------------

def calculate_abfa_accuracy(results, binary=True, threshold=1):
    """
    Calculate accuracy from results list.

    Args:
        results: list of (filename, ground_truth, predicted_count)
        binary: whether to binarize classification (e.g., count >= threshold is class 1)
        threshold: count threshold to decide positive class

    Returns:
        accuracy: float
        mismatches: list of (filename, ground_truth, predicted_count)
    """
    correct = 0
    mismatches = []

    for filename, gt, pred in results:
        pred_class = 1 if pred >= threshold else 0
        gt_class = 1 if gt >= 1 else 0 if binary else gt

        if pred_class == gt_class:
            correct += 1
        else:
            mismatches.append((filename, gt, pred))

    total = len(results)
    accuracy = correct / total if total > 0 else 0.0
    return accuracy, mismatches

# Call the function
accuracy, mismatches = calculate_abfa_accuracy(results)

# Print overall accuracy
print(f"\n📈 Overall Accuracy: {accuracy:.2%}")

# Optionally show mismatches
print("\n🔍 Mismatches:")
for filename, gt, pred in mismatches:
    print(f"{filename}: GT={gt}, Pred={pred}")



📈 Overall Accuracy: 78.60%

🔍 Mismatches:
100.png: GT=1, Pred=0
104.png: GT=1, Pred=0
104_flipped_flipped.jpg: GT=1, Pred=0
34.png: GT=1, Pred=0
35.png: GT=1, Pred=0
35_flipped.jpg: GT=1, Pred=0
35_flipped_flipped.jpg: GT=1, Pred=0
35_flipped_flipped_flipped.jpg: GT=1, Pred=0
38_flipped.jpg: GT=1, Pred=0
38_flipped_flipped_flipped.jpg: GT=1, Pred=0
39_flipped.jpg: GT=1, Pred=0
39_flipped_flipped_flipped.jpg: GT=1, Pred=0
56_flipped.jpg: GT=1, Pred=0
58_flipped.jpg: GT=1, Pred=0
58_flipped_flipped_flipped.jpg: GT=1, Pred=0
60_flipped.jpg: GT=1, Pred=0
60_flipped_flipped_flipped.jpg: GT=1, Pred=0
71_flipped.jpg: GT=1, Pred=0
71_flipped_flipped.jpg: GT=1, Pred=0
71_flipped_flipped_flipped.jpg: GT=1, Pred=0
72.png: GT=1, Pred=0
72_flipped_flipped.jpg: GT=1, Pred=0
73.png: GT=1, Pred=0
73_flipped_flipped.jpg: GT=1, Pred=0
74_flipped.jpg: GT=1, Pred=0
74_flipped_flipped_flipped.jpg: GT=1, Pred=0
75.png: GT=1, Pred=0
75_flipped.jpg: GT=1, Pred=0
75_flipped_flipped.jpg: GT=1, Pred=0
75_flippe